In [1]:
import pandas as pd 
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
#df.to_csv("Dataset/covid_toy.csv")

In [16]:
df.city.value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [17]:
df.cough.value_counts() 

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

- Fever has some missing values we will use Simple Imputer

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


- Some Categorical data One Hot Encoding and Ordinal Encoding will work fine

### Working Plane:
    1. Fill NaN values using SimpleImputer
    2. Convert Categorical Data Like Gender and City using OneHotEncoder
    3. Useing OrdinalEncoder on Cough. (Mild = 0 and Strong = 1)

In [8]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop("has_covid", axis=1), df.has_covid, test_size=0.2,random_state=42)

In [9]:
print(X_train.shape,X_test.shape)

(80, 5) (20, 5)


# Aam Jindagi :-p

In [11]:
df.columns

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')

In [26]:
# 1. Simple Imputer to fill NaN values in Fevers.
si = SimpleImputer()


X_train_fever = si.fit_transform(X_train[["fever"]])

X_test_fever = si.transform(X_test[["fever"]])

X_train_fever.shape

(80, 1)

In [27]:
#2.Using OrdinalEncoder in Cough

oe = OrdinalEncoder(categories=[["Mild","Strong"]])

X_train_cough = oe.fit_transform(X_train[["cough"]])

X_test_cough = oe.fit_transform(X_test[["cough"]])

X_train_cough.shape

(80, 1)

In [28]:
# using One Hot Encoder for categorical data

ohe = OneHotEncoder(drop="first",sparse=False)

X_train_ohe = ohe.fit_transform(X_train[["gender","city"]])

X_test_ohe = ohe.fit_transform(X_test[["gender","city"]])

X_train_ohe.shape

(80, 4)

In [30]:
# seperating Age 
X_train_age = X_train.drop(['gender', 'fever', 'cough', 'city'], axis=1).values

X_test_age = X_test.drop(['gender', 'fever', 'cough', 'city'], axis=1).values

X_train_age.shape

(80, 1)

In [33]:
# Concatinating values

X_train_final = np.concatenate((X_train_age,X_train_fever,X_train_cough,X_train_ohe),axis=1)

X_test_final = np.concatenate((X_test_age,X_test_fever,X_test_cough,X_test_ohe),axis=1)

X_train_final.shape

(80, 7)

## Mintos Jindigi

In [34]:
#Creating Transformer using Column transformer

transformer = ColumnTransformer(transformers=[
    ("simpleImpute", SimpleImputer(),["fever"]),
    ("Ordinal",OrdinalEncoder(categories=[["Mild","Strong"]]),["cough"]),
    ("Ohe",OneHotEncoder(drop='first',sparse=False),["gender","city"])
],remainder="passthrough" )

In [35]:
X_train_final = transformer.fit_transform(X_train)
X_test_final = transformer.transform(X_test)

X_train_final.shape

(80, 7)